<h1>Linear Methods - RDD-based API</h1>
<ul>
    <li>Mathematical formulation
        <ul>
            <li>Loss functions</li>
            <li>Regularizers</li>
            <li>Optimization</li>
        </ul>
    </li>
    <li>Classification
        <ul>
            <li>Linear Support Vector Machines (SVMs)</li>
            <li>Logistic regression</li>
        </ul>
    </li>
    <li>Regression
        <ul>
            <li>Linear least squares, Lasso, and ridge regression</li>
            <li>Streaming linear regression</li>
        </ul>
    </li>
    <li>Implementation (developer)</li>
</ul>

<h2>Mathematical formulation</h2>
<p>Many standard <em>machine learning</em> methods can be formulated as a <strong>convex optimization</strong> problem(凸优化问题), i.e. the task of finding a minimizer of a convex function $f$ that depends on a variable vector ${\mathbf w}$ (called <code>weights</code> in the code), which has $d$ entries.Formally, we can write this as the optimization problem $\min_{\mathbf w \in R^d} \; f(\mathbf w)$, where the objective function is of the form
$$f(\mathbf w) := \lambda R(\mathbf w) + \frac{1}{n}\sum_{i=1}^n L(\mathbf w;{\mathbf x}_i,y_i).\tag 1$$ 
Here the vectors ${\mathbf x}_i\in R^d$ are the training data examples, for $1\le i \le n$, and $y_i \in R$ are their corresponding labels, which we want to predict.We call the method <em>linear</em> if $L({\mathbf w};{\mathbf x}, y)$ can be expressed as a function of ${\mathbf w}^T x$ and $y$.Several of <code>spark.mllib</code>&#8217;s classification and regression algorithms fall into this category,and are discussed here.</p>

<p>The objective function $f$ has two parts:the <strong>regularizer</strong> that controls the complexity of the model,and the <strong>loss</strong> that measures the error of the model on the training data.The loss function $L(\mathbf w;.)$ is typically a convex function in $\mathbf w$.  The fixed regularization parameter(惩罚项,正则化参数) $\lambda \ge 0$ (<code>regParam</code> in the code) defines the trade-off(权衡) between the two goals of minimizing the loss (i.e., training error) and minimizing model complexity (i.e., to avoid overfitting).</p>

<h3>Loss functions</h3>

<p>The following table summarizes the loss functions and their gradients or sub-gradients for the
methods <code>spark.mllib</code> supports:</p>
<table>
  <thead>
    <tr><th></th><th width=280>损失函数 $L(\mathbf w; \mathbf x, y)$</th><th width=200>gradient or sub-gradient</th></tr>
  </thead>
  <tbody>
    <tr>
      <td><a href="https://en.wikipedia.org/wiki/Hinge_loss" target="_blank">hinge loss</a></td><td>$\max \{0, 1-y \mathbf w^T {\mathbf x} \},y \in \{-1, +1\}$</td>
      <td>$\begin{cases} -y\cdot \mathbf x & \text{if y $ \mathbf w^T \mathbf x \lt $ 1}, \\ 0 & \text{otherwise}.
      \end{cases}$</td>
    </tr>
    <tr>
      <td>logistic loss</td><td>$\log(1+\exp(-y{\mathbf w}^T{\mathbf x})),y \in \{-1,+1\}$</td>
      <td>$-y \left(1-\frac1{1+\exp(-y \mathbf w^T \mathbf x)} \right) \cdot \mathbf x$</td>
    </tr>
    <tr>
      <td>squared loss</td><td>$\frac{1}{2} ({\mathbf w}^T {\mathbf x}-y)^2,y \in R$</td>
      <td>$({\mathbf w}^T {\mathbf x} - y) \cdot {\mathbf x}$</td>
    </tr>
  </tbody>
</table>

<h3>Regularizers</h3>

<p>The purpose of the <a href="http://en.wikipedia.org/wiki/Regularization_(mathematics)">regularizer</a> is to encourage simple models and avoid overfitting.  We support the following regularizers in <code>spark.mllib</code>:</p>

<table>
  <thead>
    <tr><th></th><th width=200>regularizer $R(\mathbf w)$</th><th>gradient or sub-gradient</th></tr>
  </thead>
  <tbody>
    <tr>
      <td>zero(unregularized)</td><td>0</td><td>$\mathbf 0$</td>
    </tr>
    <tr>
      <td>L2</td><td>$\frac{1}{2}\|\mathbf w\|_2^2$</td><td>$\mathbf w$</td>
    </tr>
    <tr>
      <td>L1</td><td>$\|\mathbf w\|_1$</td><td>$\mathrm{sign}(\mathbf w)$</td>
    </tr>
    <tr>
      <td>elastic net</td><td>$\alpha \|\mathbf w\|_1 + (1-\alpha)\frac{1}{2}\|\mathbf w\|_2^2$</td><td>$\alpha \mathrm{sign}(\mathbf w) + (1-\alpha) \mathbf w $</td>
    </tr>
  </tbody>
</table>

<p>Here $\mathrm{sign}(\mathbf w)$ is the vector consisting of the signs ($\pm1$) of all the entries of $\mathbf w$.</p>

<p>L2-regularized problems are generally easier to solve than L1-regularized due to <a href="https://en.wikipedia.org/wiki/Smoothness" target="_blank">smoothness</a>.However, L1 regularization can help promote sparsity in weights leading to smaller and more interpretable(可解释的,能说明的) models, the latter of which can be useful for feature selection(特征选择).<a href="http://en.wikipedia.org/wiki/Elastic_net_regularization">Elastic net</a>(弹性网络) is a combination of L1 and L2 regularization. It is not recommended to train models without any regularization,especially when the number of training examples is small.</p>

<h3>Optimization</h3>

<p>Under the hood, linear methods use convex optimization methods to optimize the objective functions.<code>spark.mllib</code> uses two methods, <a target="_blank" href="https://en.wikipedia.org/wiki/Stochastic_gradient_descent">SGD</a>(Stochastic gradient descent随机梯度下降) and L-BFGS(Limited-Memory BFGS), described in the <a href="http://spark.apache.org/docs/latest/mllib-optimization.html">optimization section</a>.Currently, most algorithm APIs support Stochastic Gradient Descent (SGD), and a few support L-BFGS.Refer to <a href="http://spark.apache.org/docs/latest/mllib-optimization.html#Choosing-an-Optimization-Method">this optimization section</a> for guidelines on choosing between optimization methods.</p>

<h2>Classification</h2>

<p><a href="http://en.wikipedia.org/wiki/Statistical_classification">Classification</a> aims to divide items into
categories.The most common classification type is <a href="http://en.wikipedia.org/wiki/Binary_classification">binary classification</a>, where there are two categories, usually named positive and negative.If there are more than two categories, it is called <a href="http://en.wikipedia.org/wiki/Multiclass_classification">multiclass classification</a>.<code>spark.mllib</code> supports two linear methods for classification: linear Support Vector Machines (SVMs) and logistic regression.Linear SVMs supports only binary classification, while logistic regression supports both binary and multiclass classification problems.For both methods, <code>spark.mllib</code> supports L1 and L2 regularized variants.The training data set is represented by an RDD of `LabeledPoint` in MLlib,where labels are class indices starting from zero: $0, 1, 2, \ldots$.<font color=red>Note that</font>, in the mathematical formulation in this guide, a binary label $y$ is denoted as either $+1$ (positive) or $-1$ (negative), which is convenient for the formulation.<em>However</em>, the negative label is represented by $0$ in <code>spark.mllib</code> instead of $-1$, to be consistent with(与...一致) multiclass labeling.</p>

<h3>Linear Support Vector Machines (SVMs)</h3>

<p>The <a href="http://en.wikipedia.org/wiki/Support_vector_machine#Linear_SVM">linear SVM</a> is a standard method for large-scale classification tasks. It is a linear method as described above in equation (1), with the loss function in the formulation given by the hinge loss:</p>

<p>$$L(\mathbf w;\mathbf x,y) := \max \{0, 1-y \mathbf w^T \mathbf x \}.$$
By default, linear SVMs are trained with an L2 regularization.We also support alternative L1 regularization. In this case,the problem becomes a <a href="http://en.wikipedia.org/wiki/Linear_programming">linear program</a>.</p>

<p>The linear SVMs algorithm outputs an SVM model. Given a new data point,denoted by $\mathbf x$, the model makes predictions based on the value of $\mathbf w^T \mathbf x$.By the default, if $\mathbf w^T \mathbf x \geq 0$ then the outcome is positive, and negative otherwise.</p>

<p><strong>Examples:</strong></p>
<p>The following example shows how to load a sample dataset, build SVM model,and make predictions with the resulting model to compute the training error.</p>
<p>Refer to the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMWithSGD"><code>SVMWithSGD</code></a> and <a href="http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMModel"><code>SVMModel</code></a> for more details on the API.</p>